In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np

In [ ]:
model = YOLO("yolov8n.pt", "v8")

In [ ]:
def convert_to_yolo_format(bbox, img_width, img_height):
    start_x, start_y, end_x, end_y = bbox

    # Calculate the center coordinates of the bounding box
    x_center = (start_x + end_x) / (2 * img_width)
    y_center = (start_y + end_y) / (2 * img_height)

    # Calculate the width and height of the bounding box
    width = (end_x - start_x) / img_width
    height = (end_y - start_y) / img_height

    return x_center, y_center, width, height

In [ ]:
# Global variables
drawing = False
ix, iy = -1, -1
bbox = []
labels = []

In [ ]:
def draw_bbox(event, x, y, flags, param):
    global ix, iy, drawing, bbox, labels

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y
        bbox.append((x, y))  # Append the starting point of the new bbox

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            img_copy = img.copy()  # Create a copy to draw rectangles without filling
            for i in range(0, len(bbox), 2):
                if i+1 < len(bbox):  # Ensure there are enough elements in bbox
                    start = bbox[i]
                    end = bbox[i+1]
                    cv2.rectangle(img_copy, start, end, (0, 255, 0), 1)  # Draw the rectangle outline
            cv2.rectangle(img_copy, (ix, iy), (x, y), (0, 255, 0), 1)  # Draw the new rectangle outline
            cv2.imshow('image', img_copy)  # Show the image with updated rectangles

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        bbox.append((x, y))  # Append the ending point of the new bbox
        cv2.rectangle(img, (ix, iy), (x, y), (0, 255, 0), 1)  # Draw the final rectangle outline
        label = input("Enter label for the object: ")
        labels.append(label)
        cv2.putText(img, label, (ix, iy - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

In [ ]:
# Load your image
img = cv2.imread("images/frameforlabel.png")
clone = img.copy()

cv2.namedWindow('image')
cv2.setMouseCallback('image', draw_bbox)

while True:
    cv2.imshow('image', img)
    key = cv2.waitKey(1) & 0xFF

    # Reset the image
    if key == ord('r'):
        img = clone.copy()
        bbox = []
        labels = []

    # Save the labeled image and annotations
    elif key == ord('s'):
        img_height, img_width = img.shape[:2]
        with open("annotations.yml", "w") as f:
            for (start_x, start_y), (end_x, end_y), label in zip(bbox[::2], bbox[1::2], labels):
                x_center, y_center, width, height = convert_to_yolo_format((start_x, start_y, end_x, end_y), img_width, img_height)
                f.write(f"{label} {x_center} {y_center} {width} {height}\n")

            # Save the labeled image
            for i in range(0, len(bbox), 2):
                start = bbox[i]
                end = bbox[i+1]
                cv2.rectangle(img, start, end, (0, 255, 0), 2)
                cv2.putText(img, labels[i//2], start, cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            cv2.imwrite("labeled_image.jpg", img)
        break

    # Exit the loop
    elif key == 27:
        break

cv2.destroyAllWindows()